In [86]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import yaml
import random
import re 
from datetime import datetime


options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...")

# 로컬PC
driver = webdriver.Chrome(options=options)

# 달력에서 선택 
# 달력에서 선택 
def selectCalendar(driver , settings):

    # 달력 열기 
    driver.find_element(By.CLASS_NAME , "btn_pop.btn_d-day").click()

    target_day = settings["targetdate"]
    target_time = "18"
    year, month , day = is_valid_date(target_day) 

    time.sleep(2)
    # 현재 년,월 추출 
    now_day  = driver.find_element(By.CLASS_NAME,'date').text.split(". ")
    now_year = int(re.findall(r"\d+", now_day[0])[0])
    now_month = int(re.findall(r"\d+", now_day[1])[0])

    # 년도가 바뀌면 
    if now_month > month :
        month += 12
    
    if (month - now_month) >=3:
        raise "3개월 이상 미리 예매할 수 없습니다."

    # 원하는 달로 이동 
    for i in range(month - now_month):
        driver.find_element(By.CLASS_NAME,"slick-arrow.slick-next").click()
        time.sleep(2)
        
    # 선택 가능한 날짜 총 수가 나옴.
    CSS_SELECTOR = "tbody a[aria-disabled='false']"
    enabled_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR)
    # 출발일 선택 
    enabled_elements[day - int(enabled_elements[0].text.split("\n")[0])].click()
    time.sleep(2)
    # 출발시간 선택
    a = driver.find_element(By.CLASS_NAME, "slick-slider.slideWrap.slick-initialized")
    element = a.find_element(By.CSS_SELECTOR, f'[data-index="{target_time}"]')
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    element.click()
    
    # 달력 적용 선택 
    driver.find_element(By.CLASS_NAME,"btn_wrap").find_element(By.CLASS_NAME,"btn_bn-blue").click()

def selectPort(driver , desti : str = "서울", start = True):
    if start:
        driver.find_element(By.CLASS_NAME,"start").click()
    else:
        driver.find_element(By.CLASS_NAME,"end").click()
        
    port = driver.find_element(By.NAME,"searchTxt")
    port.click()
    port.send_keys(desti)
    port_list = driver.find_elements(By.CLASS_NAME,"sch_wrap")[0]
    port_list.click()

In [87]:
def is_valid_date(date_str: str) -> bool:
    '''
        return : year , month , day -> int 
    '''
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        split_day = date_str.split("-")

        return int(split_day[0]), int(split_day[1]), int(split_day[2])
    except ValueError:
        raise "날짜 형식이 맞지 않습니다. 2025-11-07 형태로 넣어주세요."

In [90]:
 # 달력 열기 
driver.find_element(By.CLASS_NAME , "btn_pop.btn_d-day").click()

target_day = "2025-11-19"
target_time = "18"
year, month , day = is_valid_date(target_day) 

time.sleep(2)
# 현재 년,월 추출 
now_day  = driver.find_element(By.CLASS_NAME,'date').text.split(". ")
now_year = int(re.findall(r"\d+", now_day[0])[0])
now_month = int(re.findall(r"\d+", now_day[1])[0])

# 년도가 바뀌면 
if now_month > month :
    month += 12

if (month - now_month) >=3:
    raise "3개월 이상 미리 예매할 수 없습니다."

# 원하는 달로 이동 
for i in range(month - now_month):
    driver.find_element(By.CLASS_NAME,"slick-arrow.slick-next").click()
    time.sleep(2)
    
# 선택 가능한 날짜 총 수가 나옴.
CSS_SELECTOR = "tbody a[aria-disabled='false']"
enabled_elements = driver.find_elements(By.CSS_SELECTOR, CSS_SELECTOR)
# 출발일 선택 
enabled_elements[day - int(enabled_elements[0].text.split("\n")[0])].click()
time.sleep(2)
# 출발시간 선택
a = driver.find_element(By.CLASS_NAME, "slick-slider.slideWrap.slick-initialized")
element = a.find_element(By.CSS_SELECTOR, f'[data-index="{target_time}"]')
driver.execute_script("arguments[0].scrollIntoView(true);", element)
element.click()
# 달력 적용 선택 
driver.find_element(By.CLASS_NAME,"btn_wrap").find_element(By.CLASS_NAME,"btn_bn-blue").click()

In [ ]:
a = driver.find_element(By.CLASS_NAME, "slick-slider.slideWrap.slick-initialized")
element = a.find_element(By.CSS_SELECTOR, f'[data-index="{target_time}"]')
driver.execute_script("arguments[0].scrollIntoView(true);", element)
element.click()

In [85]:
a = driver.find_element(By.CLASS_NAME, "slick-slider.slideWrap.slick-initialized")
element = a.find_element(By.CSS_SELECTOR, f'[data-index="13"]')
driver.execute_script("arguments[0].scrollIntoView(true);", element)
element.click()

In [89]:
driver.get("https://www.korail.com/ticket/search/general")
time.sleep(2)

# 출발 <-> 도착 선택
selectPort(driver, "서울" , True)
selectPort(driver, "부산" , False)


In [ ]:

time.sleep(2)
############################# 달력에서 원하는 날 선택
selectCalendar(driver , settings) #-> 당일이면 상관없음 

# 예매하기 클릭 
driver.find_element(By.CLASS_NAME , "btn_lookup").click()
time.sleep(2)
# 매크로 알림창 예방 
driver.refresh()

########################## 예매창 Scroll.... -> 기차 번호를 기준으로 진행함.
print("################# 실제 예매 서칭 .... ##########################")


user , settings = read_info()
startReservation(driver , settings)
## 20분 이내의 열차는 온라인으로 예매 불가능 -> 창구에서 해야함 

============= login finish =============
################# 실제 예매 서칭 .... ##########################


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.59)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6d57a7a05
	0x7ff6d57a7a60
	0x7ff6d55216ad
	0x7ff6d550d1c5
	0x7ff6d5531c12
	0x7ff6d55cbc67
	0x7ff6d556b068
	0x7ff6d556be93
	0x7ff6d5a629a0
	0x7ff6d5a5ce20
	0x7ff6d5a7cc15
	0x7ff6d57c309e
	0x7ff6d57cad8f
	0x7ff6d57b0be4
	0x7ff6d57b0d9f
	0x7ff6d57967f8
	0x7ff96da67374
	0x7ff96e9dcc91
